## Population
Primero creamos la clase poblacion, tendra atributos generales de toda la poblacion como el nombre, tama;o, ploidia y vida media (habran mas seguramente).

Despues se crea la clase individuo, que hereda de poblacion sus atributos y ademas tendra otros tales como el identificador, la edad, el sexo, su genotipo y quiza sus padres **ver como implementar**

### Funciones
#### init
Inicializa los atributos, que son
- **name** : nombre de la poblacion, no relevante
- **size** : tama;o de la poblacion, utilizado para recorrer posteriormente los individuos entre otras
- **ploidy** : indica si la pob esta formada por individuos haploides(1) o diploides(2)
- **vida media** : todavia sin uso, se implementara posteriormente para generar una dist de edades y para hacerla evolucionar.

Estos son los atributos que el usuario debera pasar (tiene valores por defecto), ademas se encuentran otros como:

- **gen1List,gen2List** : lista con los genotipos(alelos) posibles, ahora mismo esto da problemas porque tienen que ser accesibles por la subclase Individuals pero no lo son.
#### _getGen..
Este nuevo metodo se ha hecho para hacer accesible los atributos fuera de la clase y en la subclase pero tambien da problemas.

#### generateIndividuals
No se le pasa ningun parametro (posiblemente se cambie ya que el usuario deberia poder elegir ciertas cosas), tampoco devuelve ninguno.
Dentro, genera **indiv** que es una lista de objetos tipo **individuo** de tama;o size, ademas printa por pantalla que se ha completado el proceso
#### printIndiv
recorre la lista en un bucle y printa los atributos de cada instancia de individuo en la lista hay que **ver como hacer para printar solo un rango determinado**
#### getMeanAge
Calcula la edad media de la poblacion, de momento siempre 0
#### allGenotypes
Funcion que genera listas globales para que sean accesibles por la clase hija (probablemente haya otra forma mejor de hacerlo), estas listas contienen los alelos posibles para cada gen
#### getGenotype
Todavia no implementada, en un futuro te dira un resumen de los genotipos: frecuencia
#### summary
En un futuro, te dara un resumen de todos los atributos de la poblacion (media de edad, frecuencia de cada sexo, frecuencias alelicas, genotipicas y fenotipicas) todo esto lo hara probablemente llamando a otras funciones


In [63]:
from random import randint

#clase poblacion,atributos generales que heredara de los individuos
class Population:

    def __init__(self,size = 100,name="Homo sapiens",ploidy = 2, vida_media=55):
        self.name = name
        self.size = size
        self.ploidy = ploidy
        self.gen1List = []
        self.gen2List = []
        #ejecuta la funcion para crear las variables globales que contengan los genotipos posibles
        self.allGenotypes()
    
    #genera indivividuos
    def generateIndividuals(self):
        self.indiv = [Individual(i) for i in range(self.size)]
        print("se han generado un total de {} individuos de la poblacion {}".format(self.size,self.name))
        
    #printa individuos        
    def printIndiv(self):
        print("IDENTIF SEX AGE GENOTYPE".replace(" ","\t"))
        for x in self.indiv:
            print (x.ide,x.sex,x.age,*x.genotype.values(),sep="\t")
            
    def getMeanAge(self):
        #obtienes la edad media recorriendo la lista de individuos
        try:
            meanAge = 0
            for obj in self.indiv:
                meanAge += obj.age
            meanAge = round(meanAge/len(self.indiv),2)
            print("la edad media de la poblacion es: ",meanAge)
        except:
            print("No has inicializado la poblacion")
            
    def allGenotypes(self):
        #funcion que genera el conjunto de genotipos posibles para esta poblacion
        if self.ploidy==2:
            #genera una lista de tuplas(n=ploidy=2), cada tupla contiene un alelo
            self.gen1List = [x for x in itertools.product('Aa',repeat=2)]
            #convierte las tuplas en un string ej: ('A','a') to Aa
            self.gen1List = [''.join(elem) for elem in self.gen1List]
            #cambia el aA por Aa
            for pos,elemento in enumerate(self.gen1List):
                if elemento == 'aA':
                    self.gen1List[pos]='Aa'
            
            self.gen2List = [x for x in itertools.product('Bb',repeat=2)]
            self.gen2List = [''.join(elem) for elem in self.gen2List]
            for pos,elemento in enumerate(self.gen2List):
                if elemento == 'bB':
                    self.gen2List[pos]='Bb'
        else:
            self.gen1List = [x for x in itertools.product('Aa',repeat=1)]
            self.gen2List = [x for x in itertools.product('Bb',repeat=1)]
            
    def _getGen1List(self):
        return self.gen1List

    def _getGen2List(self):
        return self.gen2List
             
    def getGenotype(self):
        counter = {'AA':0,'Aa':0,'aa':0,'BB':0,'Bb':0,'bb':0}
        
        for elemento in self.indiv:
            for key in counter:
                if elemento.genotype['gen1'] == key or elemento.genotype['gen2']==key:
                    counter[key] += 1
        print(counter)
    def summary(self):
        pass


# Individual
Individual se trata de una clase que hereda  de population, esto es, comparte atributos (y teoricamente metodos) de su clase madre. Realmente no se si es estrictamente necesario hacer una estructura de herencia pero creo que es coherente con lo que se quiere realizar aunque todavia no esta bien implementado *falta revision* (rev 01: se ha eliminado la estructura de herencia)

Esta clase consta de los atributos y metodos necesarios para dar lugar a un individuo que formara parte de la poblacion, los atributos son: 

## atributos
- Sex: tipo string male and female
- Ide: identificador, para cada individuo (mirar de construir la lista de individuos con forma de diccionario para acceder por su identificador)
- Age: numeric int, comienza en 0
- genotype: quiza un diccionario tipo {gen1:'AA',gen2:'Bb'}, en cualquier caso se puede hacer un conjunto con todos los posibles genotipos y escoger aleatoriamente.

## Metodos

### init
Inicializa los atributos vistos anteriormente, la forma de inicializarlos puede no ser la correcta ya que en muchos se esta llamando a un metodo de la clase pero de momento funciona

### sex
Le da un valor de sexo a cada individuo de forma aleatoria al 50%, no requiere ningun parametro, devuelve male o female
### edad
Al igual que sex, asigna una edad a cada individuo. Esto habra que repensarlo porque la edad comienza en 0 para todos, dependiendo de como lo quiera implementar cuando la poblacion empiece a evolucionar esta edad debera ir aumentando para todos, algunos individuos iran muriendo = eliminando de la lista de objetos en la poblacion. Si este metodo hace correr el tiempo quiza seria mejor tenerlo en poblacion ya que el tiempo corre para todos igual *revisar*
### firstGenotipo y chooseGenotype
Genera un diccionario gen:alelos para 2 genes en total (mirar de aumentarlo mas adelante), los alelos se obtienen de la lista de alelos posibles de la clase population y se asigna uno de estos aleatoriamente si asi se quiere, si no se puede inicializar a todos los individuos con el mismo genotipo Aa/Bb. *ver como el usuario pueda pasar esta forma de generar genotipo*, seguramente desde generate individuals se deba pasar un parametro


In [64]:
#clase individuo, ya no hereda de poblacion
import itertools
import random

class Individual():

    def __init__(self,ident):
        self.sex = self.sex()
        self.ide = "id"+str(ident)
        self.age = 0
        self.genotype = self.firstGenotipo()
        
    def sex(self):
        if randint(0,1)==0:
            return "Male"
        else:
            return 'Female'
        
    def edad(self):
        #el usuario debera pasar algun parametro para indicar la dist por edades
        pass
    
    def firstGenotipo(self,homogeneous=False):
        if homogeneous:
            return {'gen1':'Aa','gen2':'Bb'}
        else:
            return self.chooseGenotype()
        
    def chooseGenotype(self):
        dict = {}

        dict['gen1']= random.choice(humanos._getGen1List())
        dict['gen2']= random.choice(humanos._getGen2List())
        
        return dict
    
    def printer(self):
        #en un principio esto no es necesario ya que no interesa conocer a un individuo concreto
        print("este individuo es {}, su sexo es {} y su genotipo es {}".format(self.ide,self.sex,self.genotype))


### Problemas:
1. Los atributos de la clase Population no son accesibles ni desde fuera (no es relevante) ni desde la clase hija Individual
2. En teoria al heredar el constructor de la clase padre deberian heredarse todos sus atributos pero no es asi, tampoco se como hacerlo o si se esta haciendo bien(por ejemplo he creado variables de clase Population dentro de esta pero fuera del constructor)
3. Idealmente, deberia poder acceder a los atributos de Population, quiza mediante Population.attribute() o de alguna otra forma pero no se como hacerlo
- Seria necesario hacer estructura de herencia? 
4. Se ha eliminado la estructura de herencia, aunque funciona lo ideal seria que para llamar a una funcion no se requiriese el nombre del objeto Population, por ejemplo ahora se le llama como humanos.funcion().

In [65]:
#se crea una nueva poblacion donde se especifican caracteristicas generales de esta
humanos = Population(size=10,name="Megadolon",ploidy=2,vida_media=23)

In [66]:
#se generan individuos en esa poblacion
humanos.generateIndividuals()

se han generado un total de 10 individuos de la poblacion Megadolon


In [67]:
humanos.printIndiv()

IDENTIF	SEX	AGE	GENOTYPE
id0	Female	0	Aa	Bb
id1	Male	0	AA	Bb
id2	Male	0	AA	Bb
id3	Male	0	aa	bb
id4	Female	0	AA	Bb
id5	Female	0	aa	Bb
id6	Female	0	Aa	bb
id7	Female	0	AA	Bb
id8	Male	0	aa	Bb
id9	Male	0	Aa	BB


In [68]:
#comienza a correr el tiempo (no implementado)
#humanos.age()

In [69]:
humanos.getMeanAge()

la edad media de la poblacion es:  0.0


In [70]:
#devuelve un diccionario con un resumen de las frecuencias absolutas para cada gen
humanos.getGenotype()

{'AA': 4, 'Aa': 3, 'aa': 3, 'BB': 1, 'Bb': 7, 'bb': 2}


In [40]:
#check if population attribute gen1List is actually a object class parameter
print(hasattr(Population,'summary'))
print(hasattr(Population,'Population.size'))
print(hasattr(Individual,'vida_media'))
print(vars(Population))

True
False
False
{'__module__': '__main__', '__init__': <function Population.__init__ at 0x0000018F39B22430>, 'generateIndividuals': <function Population.generateIndividuals at 0x0000018F39B22310>, 'printIndiv': <function Population.printIndiv at 0x0000018F39B225E0>, 'getMeanAge': <function Population.getMeanAge at 0x0000018F39B22670>, 'allGenotypes': <function Population.allGenotypes at 0x0000018F39B22700>, '_getGen1List': <function Population._getGen1List at 0x0000018F39B22790>, '_getGen2List': <function Population._getGen2List at 0x0000018F39B22820>, 'getGenotype': <function Population.getGenotype at 0x0000018F39B228B0>, 'summary': <function Population.summary at 0x0000018F39B22940>, '__dict__': <attribute '__dict__' of 'Population' objects>, '__weakref__': <attribute '__weakref__' of 'Population' objects>, '__doc__': None}


In [48]:
humanos._getGen1List()

['AA', 'Aa', 'aA', 'aa']